### taeyoonahn

**1.유니콘기업과 연금보험료의 상관관계에 대한 가정들**   
**&nbsp;&nbsp;1.1. 연금보험료가 높으면 해당 기업은 유니콘 기업일 것이다.   
&nbsp;&nbsp;1.2. 연금보험료 상승률이 높으면 해당 기업은 유니콘 기업일 것이다.**

**2. 연금보험료 측면 데이터 분석**   
**&nbsp;&nbsp;2.1. 기간 내 연금보험료 최대값이 높은 기업 분석     
&nbsp;&nbsp;2.2. 기간 내 연금보험료 평균 상승율이 높은 기업 분석**

**3. 최종산출물**   
**&nbsp;&nbsp;3.1. 기간 내 연금보험료 최대값이 높은 기업 Top n개  
&nbsp;&nbsp;3.2. 기간 내 연금보험료 평균 상승율이 높은 기업 Top n개**

In [51]:
#package 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Malgun Gothic")

from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='ignore')

In [52]:
#실수는 소수점 두자리까지 표기
pd.options.display.float_format = '{:.2f}'.format

In [53]:
#데이터 불러오기
df_ori=pd.read_csv('company_nps_data.csv',engine = "python", encoding = "CP949")
df_ori

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151
...,...,...,...,...,...,...
99796,171439,4630622,2017,2,2402740,11
99797,239388,1045021,2018,10,1428800,9
99798,272304,181898,2018,3,392560,4
99799,362194,2100218,2018,12,874780,3


In [54]:
#데이터 정보 확인
print(df_ori.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99801 entries, 0 to 99800
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   회사ID      99801 non-null  int64
 1   연매출액      99801 non-null  int64
 2   년도        99801 non-null  int64
 3   월         99801 non-null  int64
 4   월별_연금보험료  99801 non-null  int64
 5   월별_직원수    99801 non-null  int64
dtypes: int64(6)
memory usage: 4.6 MB
None


> **국민연금보험료
국민연금보험료란 국민연금 가입자들이 나중에 연금을 타기 위해 매월 내는 돈을 말한다.   
현재 보험료율은 9%로, 가입자는 매월 소득 중 9%를 보험료로 납부하고 있다.   
직장에 다니는 사람은 회사에서 절반을 내줘 실질적으로는 4.5%를 낸다.(출처 : 네이버지식백과)**  

In [55]:
#데이터 결측치 확인
df_ori.isnull()
df_ori.isnull().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
dtype: int64

In [56]:
#데이터 이상치 확인
#to_do

In [57]:
#연금보험료를 다루기 쉽게 데이터프레임 변경
df=df_ori[['회사ID','년도','월별_연금보험료']]
df

,회사ID,년도,월별_연금보험료
0,233757,2018,8912380
1,123464,2018,942380
2,342124,2018,781180
3,386420,2015,3795900
4,129243,2018,40724680
...,...,...,...
99796,171439,2017,2402740
99797,239388,2018,1428800
99798,272304,2018,392560
99799,362194,2018,874780


In [58]:
#연금보험료를 다루기 쉽게 데이터프레임 변경 
#월별 연금보험료의 결측치를 고려해 연도별 월 평균 연금보험료로 바꿈
df_year = df.groupby(['회사ID','년도']).mean()
df_year.reset_index(inplace=True)
df_year.rename(columns = {'월별_연금보험료' : '연금보험료'}, inplace = True)
df_year

,회사ID,년도,연금보험료
0,63,2018,410128228.33
1,95,2018,214837843.33
2,102,2018,173769522.50
3,295,2018,77312035.00
4,414,2018,77590193.33
...,...,...,...
10106,2400284,2018,1055497.14
10107,2437574,2018,636740.00
10108,2693684,2019,438120.00
10109,3312354,2019,1312200.00


**2. 연금보험료 측면 데이터 분석** 

**2.1. 기간 내 연금보험료 최대값이 높은 기업 분석**

In [59]:
#기간 내 연금보험료 최대값이 높은 기업 분석
df_year=df_year.sort_values(by='연금보험료',ascending=False)
df_year

,회사ID,년도,연금보험료
3793,227414,2018,945096660.00
3797,227415,2018,909976636.67
8341,419945,2019,904227043.33
3792,227414,2017,879782008.33
3796,227415,2017,864392358.33
...,...,...,...
5657,293483,2018,135026.67
9576,478999,2015,114020.00
10075,604914,2018,90000.00
5591,288014,2016,0.00


In [60]:
#기간 내 연금보험료 최대값이 높은 기업 Top n개 획득
#df_pen_top_n=df_year.head(n)
#df_pen_top_n

**2.2. 기간 내 연금보험료 평균 상승율이 높은 기업 분석**

In [61]:
#기업별 연금보험료 변화율
df_year['연금보험료_작년대비_변화율']=df_year.sort_values(['년도']).groupby(['회사ID']).연금보험료.pct_change()
df_year.dropna(inplace=True)
df_year

,회사ID,년도,연금보험료,연금보험료_작년대비_변화율
3793,227414,2018,945096660.00,0.07
3797,227415,2018,909976636.67,0.05
8341,419945,2019,904227043.33,0.10
3792,227414,2017,879782008.33,0.10
3796,227415,2017,864392358.33,0.08
...,...,...,...,...
3733,220782,2016,259710.00,-0.19
5635,291544,2018,250230.00,-0.08
9723,501043,2017,230560.00,-0.33
7356,369111,2017,204858.18,0.10


In [62]:
#기업별 기간 내 연금보험료 누적변화율
df_year['연금보험료_누적_변화율']=df_year['연금보험료_작년대비_변화율']+1
df_year['연금보험료_누적_변화율']=df_year.sort_values(['년도']).groupby(['회사ID']).연금보험료_누적_변화율.cumprod()-1
df_year

,회사ID,년도,연금보험료,연금보험료_작년대비_변화율,연금보험료_누적_변화율
3793,227414,2018,945096660.00,0.07,0.23
3797,227415,2018,909976636.67,0.05,0.19
8341,419945,2019,904227043.33,0.10,0.07
3792,227414,2017,879782008.33,0.10,0.15
3796,227415,2017,864392358.33,0.08,0.13
...,...,...,...,...,...
3733,220782,2016,259710.00,-0.19,-0.19
5635,291544,2018,250230.00,-0.08,-0.08
9723,501043,2017,230560.00,-0.33,-0.33
7356,369111,2017,204858.18,0.10,0.10


In [63]:
#예시) 회사ID 227414의 연금보험료 변화 추이 확인
df_227414 = df_year[df_year['회사ID']==227414]
df_227414

,회사ID,년도,연금보험료,연금보험료_작년대비_변화율,연금보험료_누적_변화율
3793,227414,2018,945096660.00,0.07,0.23
3792,227414,2017,879782008.33,0.10,0.15
3791,227414,2016,801098311.67,0.04,0.04


In [78]:
#기업별 기간 내 연금보험료 평균변화율
df_change=df_year.groupby(['회사ID']).연금보험료_누적_변화율.mean()
df_change=df_change.to_frame().reset_index()
df_change.rename(columns={'연금보험료_누적_변화율':'연금보험료_평균변화율'},inplace=True)
df_change.sort_values(by='연금보험료_평균변화율',ascending=False,inplace=True)
df_change

,회사ID,연금보험료_평균변화율
2443,470994,12.42
1005,233340,7.78
1925,380843,6.57
2454,472629,5.87
87,129633,5.45
...,...,...
2213,428239,-0.61
577,165510,-0.64
1095,239100,-0.66
278,140370,-0.70


In [75]:
#회사 중복 확인
len(df_change['회사ID'].unique()) == len(df_change.index)

True

In [79]:
#기간 내 연금보험료 평균 상승율이 높은 기업 Top n개 획득
#df_change_top_n=df_change.head(n)
#df_change_top_n

**3.최종산출물**

**3.1. 기간 내 연금보험료 최대값이 높은 기업 Top n개**

In [80]:
# def pensionrank() :
#     ret=x.rank(method="max")
#     return ret

In [ ]:
# df_ori.apply()

**3.2. 기간 내 연금보험료 평균 상승율이 높은 기업 Top n개**

In [ ]:
# def penchangerank() ::
#    ret=x.rank(method="max")
#    return ret

In [ ]:
# df_ori.apply()